## Problem

Build [SSVQE's cost function](https://learn.qiskit.org/course/algorithm-design/instances#instances-4-0) using the following observable:

$$
O = 2II-2XX+3YY-3ZZ
$$

This cost function should take in parameters as an input. Use the included variational form, with reference states $\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$ and weights $w_{00}=50, w_{01}=40, w_{10}= 30, w_{11}=20$.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
import numpy as np

def cost_function(theta: list[float]) -> float:
    # We've given the reference states here
    reference_circuits = []
    for i in range(2):
        for j in range(2):
            qc = QuantumCircuit(2)
            if j == 1:
                qc.x(0)
            if i == 1:
                qc.x(1)
            reference_circuits.append(qc)

    # Here's the variational form
    variational_form = TwoLocal(2, rotation_blocks=['rz', 'ry'], entanglement_blocks='cx', entanglement='linear', reps=1)

    # Start implementing the rest of SSVQE's cost function here, and observables
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

    values=[]
    weights=[50,40,30,20]
    for reference_circuit in reference_circuits:
        ansatz = reference_circuit.compose(variational_form)
        # Use estimator to get the expected values corresponding to each ansatz
        estimator = Estimator()
        job = estimator.run(ansatz, observable, theta)
        values.append(job.result().values[0])

    return np.dot(values,weights)



After you have your `cost_function`, use a classical optimizer to calculate the optimal parameters and eigenvalues for the several ansatze.

In [ ]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
import numpy as np

# Optimizing the parameters
initial_theta = np.ones(8)
optimizer = COBYLA()
optimizer_result = optimizer.minimize(fun=cost_function, x0=initial_theta)
optimal_parameters = optimizer_result.x


# Running the optimized circuit to get the eigenvalues
reference_circuits = []
for i in range(2):
    for j in range(2):
        qc = QuantumCircuit(2)
        if j == 1:
            qc.x(0)
        if i == 1:
            qc.x(1)
        reference_circuits.append(qc)

# Here's the variational form
variational_form = TwoLocal(2, rotation_blocks=['rz', 'ry'], entanglement_blocks='cx', entanglement='linear', reps=1)

# Start implementing the rest of SSVQE's cost function here, and observables
observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

eigenvalues=[]
#weights=[50,40,30,20]
for reference_circuit in reference_circuits:
    ansatz = reference_circuit.compose(variational_form)
    solution = ansatz.bind_parameters(optimal_parameters)
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator()
    job = estimator.run(solution, observable)
    eigenvalues.append(job.result().values[0])

print(eigenvalues)